In [7]:
#Libraries
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv() 

True

In [8]:
#API's
api_key = os.getenv("OPENAI_API_KEY")


# LLM Chain

In [9]:
#LLM
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instanc
llm.invoke("who is Valentin Valencia?")



AIMessage(content='There is no widely known or notable figure named Valentin Valencia. It is possible that Valentin Valencia is a private individual or a fictional character.')

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class personal assistant."),
    ("user", "{input}")
])

In [11]:
chain = prompt | llm 

In [13]:
chain.invoke({"input": "who is Valentin Valencia?"})

AIMessage(content="I'm sorry, but I couldn't find any specific information on a person named Valentin Valencia. It is possible that this individual is not widely known or does not have a public presence. If you have more context or details, I may be able to provide more relevant information.")

In [14]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [15]:
chain = prompt | llm | output_parser

In [16]:
chain.invoke({"input": "who is Valentin Valencia?"})

"I'm sorry, but I couldn't find any specific information about a person named Valentin Valencia. Could you provide more context or details to help me assist you better?"

# Retrieval Chain

In [17]:
"""
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://en.wikipedia.org/wiki/President_of_Colombia")

docs = loader.load()
"""
# Load content from the local file
file_path = "text.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_content = file.read()

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [20]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents([Document(page_content=text_content)])
vector = FAISS.from_documents(documents, embeddings)

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [23]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [24]:
response = retrieval_chain.invoke({"input": "who is Valentin Valencia?"})
print(response["answer"])

Valentin Valencia is a trilingual communicator and full-stack developer currently pursuing a degree in Systems and Computing Engineering at Universidad Tecnológica de Pereira. He has a strong foundation in academic excellence and has also studied at L'École de Technologie Supérieure in Montreal, Canada. Valentin is passionate about using technology to drive innovation and values collaboration, honesty, consistency, respect, and responsibility in his work. He is also a Taekwondo practitioner and cycling enthusiast who enjoys exploring different cultures, languages, and technologies.


# Conversation Retrieval Chain

In [27]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user is asking a follow up question.

In [28]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can you tell me about Valentin Valencia?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "where was he born"
})

[Document(page_content="Summary\n\n👋 Hello, I'm Valentin Valencia, and I believe in the power of connecting people, ideas, and technology. 🌐\n\n📚 With a foundation in academic excellence, I graduated with honors from Colegio Salesiano San Juan Bosco and embarked on a journey of continuous learning. Currently, I'm pursuing a degree in Systems and Computing Engineering at Universidad Tecnológica de Pereira, where I'm passionate about harnessing technology to drive innovation.\n\n🌍 My horizons expanded during an eye-opening academic exchange at L'École de Technologie Supérieure in Montreal, Canada, where I delved into the realm of génie en technologies de l'information.\n\n💬 I'm not just proficient in one language; I'm a trilingual communicator. Fluent in Spanish, English (certified at C1 level by the British Council's APTIS test), and French (certified as DALF C1 by L'Alliance Française), I bridge cultural gaps and collaborate seamlessly on a global scale.\n\n💻 My skills shine in the rea

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [35]:
chat_history = [HumanMessage(content="Hello"), AIMessage(content="Hello!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "who is Valentin Valencia?"
})

{'chat_history': [HumanMessage(content='Hello'), AIMessage(content='Hello!')],
 'input': 'who is Valentin Valencia?',
 'context': [Document(page_content="Summary\n\n👋 Hello, I'm Valentin Valencia, and I believe in the power of connecting people, ideas, and technology. 🌐\n\n📚 With a foundation in academic excellence, I graduated with honors from Colegio Salesiano San Juan Bosco and embarked on a journey of continuous learning. Currently, I'm pursuing a degree in Systems and Computing Engineering at Universidad Tecnológica de Pereira, where I'm passionate about harnessing technology to drive innovation.\n\n🌍 My horizons expanded during an eye-opening academic exchange at L'École de Technologie Supérieure in Montreal, Canada, where I delved into the realm of génie en technologies de l'information.\n\n💬 I'm not just proficient in one language; I'm a trilingual communicator. Fluent in Spanish, English (certified at C1 level by the British Council's APTIS test), and French (certified as DALF

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!

In [36]:
print(chat_history)

[HumanMessage(content='Hello'), AIMessage(content='Hello!')]


In [37]:
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "list his universitites"
})

{'chat_history': [HumanMessage(content='Hello'), AIMessage(content='Hello!')],
 'input': 'list his universitites',
 'context': [Document(page_content='Looking forward to connecting and embarking on exciting journeys together! 🚀🌎 #OpenToOpportunities #TechEnthusiast #LanguageLover #TeamPlayer\n\n\nEducation\nUniversidad Tecnológica de Pereira logo\nUniversidad Tecnológica de PereiraUniversidad Tecnológica de Pereira\nBachelor of Engineering - BE, Computer Software EngineeringBachelor of Engineering - BE, Computer Software Engineering\nJan 2019 - Nov 2024Jan 2019 - Nov 2024\nA university community committed to the human and academic development of citizens with critical thinking and the ability to participate in strengthening democracy; with an interdisciplinary perspective for understanding and seeking solutions to societal problems, grounded in the knowledge of sciences, disciplines, arts, and wisdom.A university community committed to the human and academic development of citizens wit

In [38]:
print(chat_history)

[HumanMessage(content='Hello'), AIMessage(content='Hello!')]


In [39]:
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "list his hobbies"
})

{'chat_history': [HumanMessage(content='Hello'), AIMessage(content='Hello!')],
 'input': 'list his hobbies',
 'context': [Document(page_content='Looking forward to connecting and embarking on exciting journeys together! 🚀🌎 #OpenToOpportunities #TechEnthusiast #LanguageLover #TeamPlayer\n\n\nEducation\nUniversidad Tecnológica de Pereira logo\nUniversidad Tecnológica de PereiraUniversidad Tecnológica de Pereira\nBachelor of Engineering - BE, Computer Software EngineeringBachelor of Engineering - BE, Computer Software Engineering\nJan 2019 - Nov 2024Jan 2019 - Nov 2024\nA university community committed to the human and academic development of citizens with critical thinking and the ability to participate in strengthening democracy; with an interdisciplinary perspective for understanding and seeking solutions to societal problems, grounded in the knowledge of sciences, disciplines, arts, and wisdom.A university community committed to the human and academic development of citizens with crit

In [41]:
print(chat_history)

[HumanMessage(content='Hello'), AIMessage(content='Hello!')]
